In [1]:
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

sns.set_theme()

# Q1

In [2]:
df_jan_2023 = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")
print(df_jan_2023.shape)
df_jan_2023.head()

(3066766, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Answer: 19 columns.

# Q2

In [3]:
# add duration column
df_jan_2023["duration"] = df_jan_2023["tpep_dropoff_datetime"] - df_jan_2023["tpep_pickup_datetime"]
# get duration in minutes
df_jan_2023["duration"] = df_jan_2023["duration"].apply(lambda td: td.total_seconds() / 60)
# df_jan_2023 = df_jan_2023[df_jan_2023["duration"] > 0]

In [4]:
# Answer
df_jan_2023["duration"].describe()['std']

42.59435124195458

# Q3

In [5]:
# keep only the records where the duration was between 1 and 60 minutes (inclusive)
before_cleaning = df_jan_2023.shape[0]
df_jan_2023 = df_jan_2023[(df_jan_2023["duration"] >= 1) & (df_jan_2023["duration"] <= 60)]
after_cleaning = df_jan_2023.shape[0]

In [6]:
# Answer
after_cleaning / before_cleaning

0.9812202822125979

# Q4

In [7]:
categorical = ["PULocationID", "DOLocationID"]
df = df_jan_2023[categorical].astype(str)

# turn df into list of dictionaries
train_dict = df.to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)

# Answer
X_train.shape[1]

515

# Q5

In [8]:
y_train = df_jan_2023["duration"].values
model = LinearRegression(n_jobs=-1)
model.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [9]:
y_pred = model.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

/opt/conda/envs/mlops-zoomcamp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261932106969

# Q6

In [10]:
df_fev_2023 = pd.read_parquet("data/yellow_tripdata_2023-02.parquet")
df_fev_2023["duration"] = df_fev_2023["tpep_dropoff_datetime"] - df_fev_2023["tpep_pickup_datetime"]
df_fev_2023["duration"] = df_fev_2023["duration"].apply(lambda td: td.total_seconds() / 60)
df_fev_2023 = df_fev_2023[(df_fev_2023["duration"] >= 1) & (df_fev_2023["duration"] <= 60)]

In [14]:
df_val = df_fev_2023[categorical + ["duration"]].astype(str)
# turn df into list of dictionaries
X_val = dv.transform(df_val.to_dict(orient='records'))
y_val = df_val["duration"].values

In [15]:
y_pred = model.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/opt/conda/envs/mlops-zoomcamp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811818743246608